# Regression model for Taxi fares using NimbusML

Regression is a ML task type of supervised machine learning algorithms. A regression ML model predicts continuous value outputs (such as numbers). For instance, predicting the fare of a Taxi trip or predicting the price of a car is a regression problem.

## Verify your NimbusML version

In [ ]:
import nimbusml
from nimbusml import FileDataStream, DataSchema
print("nimbusml version: ", nimbusml.__version__)

## Load your data

In [ ]:
# These are actually faster than Pandas Dataframes
# Do you want to know more? https://arxiv.org/pdf/1905.05715.pdf
ds_train = FileDataStream.read_csv("../datasets/taxi/taxi-fare-train.csv")
ds_test = FileDataStream.read_csv("../datasets/taxi/taxi-fare-test.csv")
ds_train.head(5)

## Verify the schema of your data 

In [ ]:
# Want to know more about the Schema?
# https://docs.microsoft.com/en-us/nimbusml/concepts/schema

# Want to know more about data types? in Nimbus?
# https://docs.microsoft.com/en-us/nimbusml/concepts/types
ds_train.schema

#### This is the infered schema. Does this look right to you?

In [ ]:
# I'm changing the type of some columns
# the defaults are not smart, I know more of the problem...
ds_train.schema['RateCode'].type = "TX"
ds_train.schema['PassengerCount'].type = "R4"
ds_train.schema['TripTime'].type = "R4"
ds_train.schema['TripDistance'].type = "R4"
ds_train.schema[6].type = "R4"

In [ ]:
ds_train.schema

In [ ]:
# you can also set it at load time:
# schema = DataSchema.read_schema("./datasets/taxi/taxi-fare-train.csv", collapse = False, sep = ',')
#modify then load
ds_test = FileDataStream.read_csv("../datasets/taxi/taxi-fare-test.csv",  schema=ds_train.schema  )

In [ ]:
ds_test.schema

## Data transformations pipeline for NimbusML model

In [ ]:
from nimbusml import Pipeline
from nimbusml.feature_extraction.categorical import OneHotVectorizer
from nimbusml.preprocessing.normalization import MeanVarianceScaler

# https://docs.microsoft.com/en-us/python/api/nimbusml/nimbusml.feature_extraction.categorical.onehotvectorizer?view=nimbusml-py-latest
onv = OneHotVectorizer()        <<['VendorId','RateCode','PaymentType']
mvs = MeanVarianceScaler()      <<['PassengerCount','TripTime','TripDistance']

preprocess_pipeline = Pipeline([onv,mvs])

## Make it a training pipeline

In [ ]:
from nimbusml.linear_model import OrdinaryLeastSquaresRegressor 
# OLS likely has high bias on this data
# probably FastTree or LightGbm are better...

olsr = OrdinaryLeastSquaresRegressor(feature= ['VendorId', 'RateCode', 'PassengerCount', 'TripTime','TripDistance', 'PaymentType'] , label='FareAmount')

training_pipeline = preprocess_pipeline.clone()

In [ ]:
training_pipeline.append(olsr)

## Fit both pipelines

In [ ]:
# fit the pipelines
preprocess_pipeline.fit(ds_train, 'y')
training_pipeline.fit(ds_train, 'y')

print("preprocess pipeline:", preprocess_pipeline)
print("training pipeline:", training_pipeline)

## Observe the transformed data

In [ ]:
preprocess_pipeline.transform(ds_train)

## Measure the performance on the training and test set

In [ ]:
metrics, scores = training_pipeline.test(ds_train, output_scores=True)
#print(scores) # uncomment this if you want to look at the scores
metrics

In [ ]:
metrics, scores = training_pipeline.test(ds_test, output_scores=True)
#print(scores) # uncomment this if you want to look at the scores
metrics

## Visualize the pipeline

In [ ]:
### Does this not work?
### make sure you've installed graphviz https://graphviz.gitlab.io/download/
### make sure you've run set_path_graphviz.bat 
from nimbusml.utils.exports import img_export_pipeline
figure = img_export_pipeline(training_pipeline,ds_train)
figure

## Save the model

In [ ]:
print("Saving the ML.NET Model as a file...")

model_file_path = "../models/ml_net_taxi_python.zip"
training_pipeline.save_model(model_file_path)

print("The model was saved to: {}".format(model_file_path))